# Demos: Lecture 6

## Demo 1: teleportation 

 <img src="fig/teleportation.png" width="800px">

In [7]:
import pennylane as qml

def create_entangled_state(wires):
    qml.Hadamard(wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])

dev = qml.device('default.qubit', wires=3)

def alice_prepare_state():
    qml.PauliZ(wires=0)
    qml.PauliX(wires=0)
    qml.RX(-3, wires=0)
    qml.Hadamard(wires=0)
    qml.RY(600, wires=0)

@qml.qnode(dev)
def look_at_state():
    alice_prepare_state()
    return qml.probs(wires=0)

In [8]:
look_at_state()

tensor([0.47812985, 0.52187015], requires_grad=True)

In [17]:
@qml.qnode(dev)
def teleportation():
    # Prepare the qubits
    alice_prepare_state()
    create_entangled_state(wires=[1, 2])

    # Basis rotation
    qml.adjoint(create_entangled_state)(wires=[0, 1])

    # Alice measures her qubits
    m1 = qml.measure(wires=0)
    m2 = qml.measure(wires=1)

    # Bob applies correction
    qml.cond(m2 == 1, qml.PauliX)(wires=2)
    qml.cond(m1 == 1, qml.PauliZ)(wires=2)

    return qml.probs(wires=2)

In [18]:
teleportation()

tensor([0.47812985, 0.52187015], requires_grad=True)

## Demo 2: expectation values

In [21]:
import numpy as np

In [24]:
dev = qml.device("default.qubit", wires=1, shots=1000)

@qml.qnode(dev)
def circuit():
    qml.RX(2*np.pi/3, wires=0)
    return qml.sample(qml.PauliZ(0))

In [26]:
np.mean(circuit())

-0.494

In [29]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def circuit():
    qml.RX(2*np.pi/3, wires=0)
    return qml.expval(qml.PauliY(0))

In [30]:
circuit()

tensor(-0.8660254, requires_grad=True)